In [ ]:
import glob
import json
import os
import pandas as pd

def get_method_df(method: str, n: int) -> pd.DataFrame:

    method_df = pd.DataFrame()
    for fp in sorted(glob.glob(os.path.join("../results/", method, f"n={n}/", "l=*.json"))):
        with open(fp, "r") as f:
            result_json = json.load(f)
        if len(result_json["results"]) == 0:
            continue
        result_df = pd.DataFrame(result_json["results"])
        result_df["l"] = result_json["l"]
        method_df = pd.concat([method_df, result_df])
    
    return method_df

In [ ]:
import plotly.graph_objects as go

method = "modal"
ns = [10, 20, 30, 40, 100]

fig = go.Figure()
skips = None

for n in ns:
    df = get_method_df(method=method, n=n)
    p_trend = [len(df[(df.result == 1) & (df.l == l)]) / 100 for l in sorted(df.l.unique())]
    fig.add_trace(go.Scatter(x=sorted(df.l.unique() / n), y=p_trend, name=n))
    if skips is None:
        skips = [(100 - len(df[df.l == l])) / 100 for l in sorted(df.l.unique())]

if sum(skips) > 0:
    fig.add_trace(go.Scatter(x=sorted(df.l.unique() / n), y=skips, name="skips"))

fig.update_layout(
    title="Probability of formula satisfiability",
    xaxis_title="Ratio",
    yaxis_title="Probability",
    legend_title="# of variables"
)
fig.write_image(f"../plots/prob_{method}.png")
fig.show()